In [3]:
import cobra
from cobra.core import Metabolite, Reaction, Model
from d3flux import flux_map
import cobra.test
import requests
import hashlib
import re
import numpy as np

In [4]:
def get_sum_fluxes(metabolite):
    summarylines=str(metabolite.summary()).split("\n")
    stop=0
    if summarylines[6]=="Empty DataFrame":
        print("No producing reactions")
        return 0
    i=0
    for line in summarylines[7:] :
        i+=1
        if line =="Consuming Reactions" :
            stop=i-2
    sommefluxes=0.0
    for line in summarylines[7:7+stop] :
        flux=float(line.split()[1])
        sommefluxes+=flux
    return sommefluxes
def get_fluxes_from_mitocore_metabolite(metabolite):
    """
    fonction qui a partir d'un métabolite, récupère tout les flux producteurs du métabolites, et
    fais la somme des flux par sous système 
    """
    summarylines=str(metabolite.summary()).split("\n")
    stop=0
    i=0
    dictflux=dict()
    if summarylines[6]=="Empty DataFrame":
        print("No producing reactions")
        return 0
    for line in summarylines[7:] :
        i+=1
        if line =="Consuming Reactions" :
            stop=i-2
    sommefluxes=0.0
    for line in summarylines[7:7+stop] :
        flux=float(line.split()[0].strip("%"))
        sommefluxes+=flux
        r_id=line.split()[2]
        subsystem=mitocore_mouse.reactions.get_by_id(r_id).notes['SUBSYSTEM']
        if subsystem not in dictflux.keys():
            dictflux[subsystem]=0
        dictflux[subsystem]+=flux
    print(dictflux)
    return sommefluxes
def get_mitocore_respiratory_exchange_ratio(model) :
    flux_model_o2=model.reactions.O2t.flux
    flux_model_Co2=abs(model.reactions.CO2t.flux)
    print("o2 : "+str(flux_model_o2))
    print("Co2 : "+str(flux_model_Co2))
    return flux_model_Co2/flux_model_o2

In [5]:
mitocore_mouse=cobra.io.read_sbml_model("Models/Mitocore_mouse.xml")

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [7]:
#condition normale : 
mitocore_mouse.objective=mitocore_mouse.reactions.OF_ATP_MitoCore
solution=mitocore_mouse.optimize()
print(solution.objective_value)
healthy_model_objective_value=solution.objective_value
compteur=0
for f in solution.fluxes :
    if f != 0 :        
        compteur+=1
print("Nombre de réactions utilisées pour maximiser la production d'ATP : "+str(compteur))  
print(get_fluxes_from_mitocore_metabolite(mitocore_mouse.metabolites.accoa_m))
print(mitocore_mouse.metabolites.accoa_m.summary())
print(mitocore_mouse.reactions.LDH_L.summary())

100.89230387215642
Nombre de réactions utilisées pour maximiser la production d'ATP : 226
{'Isoleucine degradation': 0.06, 'FA and ketone body metabolism / Ketogenesis': 19.93, 'Ketogenesis / Leucine degradation': 0.24, 'FA metabolism': 41.459999999999994, 'TCA cycle': 38.35}
100.03999999999999
accoa_m
Formula: C23H34N7O17P3S

Producing Reactions
-------------------
Percent   Flux        Reaction                                                                  Definition
  0.06%  0.004        ACACT10m                                     2maacoa_m + coa_m --> accoa_m + ppcoa_m
 19.93%  1.355        ACACT1rm                                             2.0 accoa_m <=> aacoa_m + coa_m
  0.24%  0.016           HMGLm                                               hmgcoa_m --> acac_m + accoa_m
  6.91% 0.4697 MTPC14_MitoCore 3tetd7ecoa_m + coa_m + h2o_m + nad_m <=> accoa_m + ddcacoa_m + h_m + nadh_m
  6.91% 0.4697 MTPC16_MitoCore 3hexdcoa_m + coa_m + h2o_m + nad_m <=> accoa_m + h_m + nadh_m + t

In [7]:
#modification des réactions par rapport à la littérature
###############################################################################
list_modified_reactions=list()
count=0
with open("model_convertion.csv","r") as f :
    f.readline()
    line=f.readline()
    while line!="":
        data=line.split(",")
        gene=data[0].strip('"')
        mitocore_id=data[1].strip('"').strip()
        #print(mitocore_id)
        fold_change=data[2].strip('"')
        multiplicateur=data[3].strip('"')
        if multiplicateur == "Hausse":
            multiplicateur=2.0
        if multiplicateur=="Baisse" :
            multiplicateur=0.5
        if multiplicateur=="Warning" :
            multiplicateur=0.5
            
    
        if mitocore_id!="None" :
            
            list_r_id=mitocore_id.split(";")
            
            for rid in list_r_id :
                list_modified_reactions.append(rid)
                #print(rid)
                r=mitocore_mouse.reactions.get_by_id(rid)
                print(r)
                print(r.upper_bound)
                print(r.lower_bound)
                print(multiplicateur)
                
                print(r.flux)
                if r.flux != 0.0 :
                    print(float(r.flux)*float(multiplicateur))
                    r.upper_bound=max(float(r.flux)*float(multiplicateur),0)
                print(r.upper_bound)
                count+=1
                
        line=f.readline()
print(count)

ICDHy: icit_c + nadp_c <=> akg_c + co2_c + nadph_c
1000.0
-0.1
3.226567036888505
-0.09600000000377494
-0.3097504355534766
0
FBP: fdp_c + h2o_c --> f6p_c + pi_c
0.0
0.0
3.2716082342311243
0.0
0.0
ALCD2if: etoh_c + nad_c --> acald_c + h_c + nadh_c
1000.0
0.0
4.027822200226875
0.0
1000.0
AKGDm: akg_m + coa_m + nad_m --> co2_m + nadh_m + succoa_m
1000.0
0.0
4.141059695365509
6.822391304346884
28.25192965644301
28.25192965644301
MTHFD2m: mlthf_m + nad_m <-- methf_m + nadh_m
0.0
-1000.0
4.531535541183195
0.0
0.0
MTHFDm: mlthf_m + nadp_m <-- methf_m + nadph_m
0.0
-1000.0
4.531535541183195
0.0
0.0
CSm: accoa_m + h2o_m + oaa_m --> cit_m + coa_m + h_m
1000.0
0.0
0.5
6.801391304346884
3.400695652173442
3.400695652173442
r0722: HC01401_m + nad_m <=> 3oddcoa_m + h_m + nadh_m
1000.0
-1000.0
0.5
0.46967391304347994
0.23483695652173997
0.23483695652173997
r0726: HC01403_m + nad_m <=> 3odcoa_m + h_m + nadh_m
1000.0
-1000.0
0.5
0.46967391304347994
0.23483695652173997
0.23483695652173997
r0730: HC01405_m

In [8]:
#condition malade : 
mitocore_mouse.objective=mitocore_mouse.reactions.OF_ATP_MitoCore
solution=mitocore_mouse.optimize()
print(solution.objective_value)
sick_model_objective_value=solution.objective_value
compteur=0
for f in solution.fluxes :
    if f != 0 :        
        compteur+=1
print("Nombre de réactions utilisées pour maximiser la production d'ATP : "+str(compteur))  
print(get_fluxes_from_mitocore_metabolite(mitocore_mouse.metabolites.accoa_m))
print(mitocore_mouse.metabolites.accoa_m.summary())
print("Pourcentage d'atp produit dans le modèle malade par rapport au modèle sain :\n")
print(str(sick_model_objective_value/healthy_model_objective_value*100)+" %")

50.73069360274499
Nombre de réactions utilisées pour maximiser la production d'ATP : 239
{'Isoleucine degradation': 0.06, 'FA and ketone body metabolism / Ketogenesis': 13.29, 'FA metabolism': 33.18, 'TCA cycle': 53.48}
100.01
accoa_m
Formula: C23H34N7O17P3S

Producing Reactions
-------------------
Percent  Flux        Reaction                                                                  Definition
  0.06% 0.002        ACACT10m                                     2maacoa_m + coa_m --> accoa_m + ppcoa_m
 13.29% 0.452        ACACT1rm                                             2.0 accoa_m <=> aacoa_m + coa_m
  5.53% 0.188 MTPC14_MitoCore 3tetd7ecoa_m + coa_m + h2o_m + nad_m <=> accoa_m + ddcacoa_m + h_m + nadh_m
  5.53% 0.188 MTPC16_MitoCore 3hexdcoa_m + coa_m + h2o_m + nad_m <=> accoa_m + h_m + nadh_m + tetd7ecoa_m
 53.48% 1.819            PDHm                          coa_m + nad_m + pyr_m --> accoa_m + co2_m + nadh_m
  5.53% 0.188           r0287                                   

### analyse rapide du modèle C3KO
production d'atp divisée par 2 (due à la division par 2 des réactions sans valeurs exactes) </br>  </br>
switch sur le métabolisme glycolytique comme attendu selon kramerova 2016 :"These data suggest that the glycolytic pathway is up-regulated in C3KO muscles to compensate for defects in lipid metabolism.
C3KO"  </br>  </br>

239 réactions utilisées pour maximiser la production d'atp au lieu de 216 : signe de l'utilisation d'autres voies métaboliques ?
##### sans toucher au réactions qui ne sont baissées que parce que le nb de mitochondrie baisse
77% de production d'atp, baisse glycolyse, hausse lipidique


In [7]:
print(mitocore_mouse.reactions.CSm.flux)
print(mitocore_mouse.reactions.CSm.upper_bound)

6.801391304346884
6.801391304346884


In [9]:
#condition malade : 
mitocore_mouse.objective=mitocore_mouse.reactions.OF_ATP_MitoCore
solution=mitocore_mouse.optimize()
print(solution.objective_value)
compteur=0
for f in solution.fluxes :
    if f != 0 :        
        compteur+=1
print("Nombre de réactions utilisées pour maximiser la production d'ATP : "+str(compteur))  
print(get_fluxes_from_mitocore_metabolite(mitocore_mouse.metabolites.accoa_m))
print(mitocore_mouse.reactions.CSm.summary())

50.73069360274499
Nombre de réactions utilisées pour maximiser la production d'ATP : 239
{'Isoleucine degradation': 0.06, 'FA and ketone body metabolism / Ketogenesis': 13.29, 'FA metabolism': 33.18, 'TCA cycle': 53.48}
100.01

CSm
===
accoa_m + h2o_m + oaa_m --> cit_m + coa_m + h_m
Bounds: 0.0, 3.400695652173442
Flux: 3.401



### Modèle sain en exercice

selon littérature : </br>
*  entrée de glucose dans muscle squelettique : 86+-11 umol/min/kg = 0.09 umol/min/gDW (PMID: 1601987)
0.3-0.5 to 5.2-5.4 mumol/min per 30g chez le rat : = 0.016 µmol.g-1.min-1
https://doi.org/10.1152/physiol.00012.2005 : 
low intensity exercice : 1.2mmol.min-1/leg = 1200.min-1/leg = 1200.min-1.8.31kg-1 =
1200.min-1.1.96852300242 kgDw-1 = 609.594095941µmol.min-1.kgDw-1 = 0.6 µmol.min-1.gDw-1
medium intensity exercice : 2.4mmol.min-1/leg = 2400µmol.min-1/leg = 2400µmol.min-1.8.31kg-1 =
2400µmol.min-1.1.96852300242 kgDw-1 = 1219.18819188µmol.min-1.kgDw-1 = 1.219 µmol.min-1.gDw-1
 high intenisty exercice : 4mmol.min-1/leg = 4000mmol.min-1/leg = 4000µmol.min-1.8.31kg-1 =
 4000µmol.min-1.1.96852300242 kgDw-1 = 2031.9803198µmol.min-1.kgDw-1=2.031µmol.min-1.gDw-1

* entrée de d'acide gras libre dans muscle squelettique (fémoral) : The femoral muscle FFA
uptake rates in this study in the fasting state (0.43 µmol*100g-1*min-1) = 0.043 µmol* g-1 *min-1
(PMID: 10342811)

* entrée de lipide dans un cellule de COEUR 5.58±2.02 μmol · 100 g−1 · min−1 = 0.077 µmol/g-1/min-1 : (pmid :17088453)
*

dans les modèles de bases : </br> 
* entrée de glucose : 0.9 GLCt1r
* entrée de lipides : 1 HDCAtr
*  glucose+lipides=100%=1.9 </br> </br>
exercice à 25% de Vo^2 : </br>
* pourcentage de glucose : 11% environ
* pourcentage de lipides : 89% environ : hargreaves 2020
1.9*11%=0.209, 1.9*89%=1.690 </br> </br>
exercice à 65% de la Vo^2 max </br>
* contribution totale = 2.33*contribution à 25% 1.9*2.33=4.427
*  pourcentage glucose : 45.807% 4.427*45.807/100= 2.027
* pourcnetage lipides : 54.193% 4.427*54.193/100=2.400 </br> </br>
exercice à 85% de la Vo2 max: 
*contribution totale =  3.41*contribution à 25% 1.9*3.41= 6.479
* pourcentage glucose = 75.562% 6.479*75.562/100=4.895
* pourcentage lipide = 24.438%  6.479*24.438/100=1.582

#### Autre entrées de métabolites :
##### oxygène : 
The maximal oxygen uptake of skeletal muscle is around 300-400 mL min-1 kg-1 (PMID: 10759582):
0.4 L = 0.4*1/22.4 mol = 0.0.01785714285 mol
 0.01785 mol*min-1*kg-1 = 1785 umol*min-1*kg-1 = 17.85umol*min-1*g-1
 (dry weight :1785 µmol*min-1*kg-1=1785 µmol*min-1*0.2421307506 KGdW=7372.05000016 µmol*min-1*KGdW-1=73.72 µmol*min-1*gDw-1)
 
at rest : 0.14 ml/100ML muscle / min = 0.14ml/110gmuscle/min=0.00127272727ml/gmuscle/min=1e^-6L/gmuscle/min=
        4.46428571e-8mol/gmuscle/min=0.04µmol/gmuscle/min (https://doi.org/10.1152/ajpheart.00996.2010)
durant exercice : 5ml/100ml/min = 5ml/110g/min=0.045ml/g/min= 4.5e-5L/g/min=1.875e-6mol/g/min=4.5µmol/g/min


3-hudroybutanoate : DOI : 10.1055/s-2007-1003386
at rest :50 µmol/L dialysat = 50µmol 1.6g muscle = 50µmol/min/0.39 gDw=19.5
exercice : 

acetoacetate : 1.62 +- 0.03 mg per gm per hour

 #### Production
ATP : 
Les valeurs fluctuent entre 22.5 et 25 µmol/min/g pour des exercices montant jusqu'a 90% VO2max: DOI: 10.1097/00005768-200004000-00007 · Source: PubMed 
consommation augmente -> donc production augmente
atp utilisation by muscle :
During submaximal exercise at ~200 W (~75% VO2 max), the corresponding values are 0.4 mmol ATP kg−1 s−1 = 2.4µmol/min/g : https://doi.org/10.1038/s42255-020-0251-4 </br>
 see also : bangsbo 2001 et bangsbo 2000</br>
 
 mean mass of leg muscle : 8.31 kg https://doi.org/10.1093/gerona/glv136
 
 """The density of muscle is 1.1 g/ml. Using the averages, 1 lite"(r of muscle weights 1.06 kg  . ATPase consume 6.2-18.7 µmol/min/g : 111-332 µmol/L/s=6660-19920µmol/L/min=6283.01886792-18792.4528302µmol/Kg/min=6.2-18.7µmol/g/min : (PMC2278479)
 </br></br>
 22.5+6.2=29.1
25+17.3=42.3"""

 
 lactate :flux estimé au repos de la LDH : 0 , à 65% VO2max : 1.07 (5.2*4.13=1.07 : WW to DW)      DOI:10.1097/00005768-200004000-00007 (Besoin d'ajouter du lactate au milieu ?)</br> 
 
 à 65% production de lactate : 3mmol/min/2.5kg = 3000 µmol/min/2.5kg=3µm/min/2.5g=7.5µm/min/g
 à 85% production de lactate : environ 10mmol/min/2.5kg = 10000 µmol/min/2.5kg=10µmol/min/2.5g=25µmol/min/g
 
 At rest, the systemic lactate Ra of the athletes (18+-3 µmol/kg body mass/min)DOI: 10.1152/ajpendo.00273.2002 · Source: PubMed

The tracer-calculated lactate uptake and release were 0.16–0.18 mmol/min at rest under both conditions
 
 
 co2 :</br>
 oxygène :</br>
https://journals.physiology.org/doi/full/10.1152/ajpheart.00996.2010 : </br>

acetyl-coa :doi:10.1152/ajpregu.1998.275.2.r418 </br> 
 at rest : ~12-18 µmol/g/min 
             65% : ~20-25 µmol/g/min
 
 ### wet to dry
 https://www.researchgate.net/figure/Muscle-wet-to-dry-weight-ratios-Mean-SD_tbl2_11768742
 W/D = 4.13
 

In [23]:
mitocore_mouse=cobra.io.read_sbml_model("Models/Mitocore_mouse.xml")
#mitocore_mouse.reactions.O2t.upper_bound=1000 # doit-on lever la contrainte de l'oxygène ?
mitocore_mouse.reactions.O2t.upper_bound=18 # doit-on adapter la contrainte à un muscle squelettique ?
#mitocore_mouse.reactions.O2t.upper_bound=74.34 #valeur en poid sec#
#mitocore_mouse.reactions.L_LACt2r.upper_bound=0 # pas d'entrée de lactate
#mitocore_mouse.reactions.O2t.upper_bound=4.5# valeur littérature au repos

In [24]:
#25% VO2max
#coeur
#mitocore_mouse.reactions.GLCt1r.upper_bound=0.209
#mitocore_mouse.reactions.HDCAtr.upper_bound=1.690
#muscle si on laisse les valeurs bruts trouvées dans la littérature
#mitocore_mouse.reactions.GLCt1r.upper_bound=0.09
#mitocore_mouse.reactions.HDCAtr.upper_bound=0.043
#muscle si on réorganise selon les pourcentages de la littérature
mitocore_mouse.reactions.GLCt1r.upper_bound=(0.09+0.043)*0.11
mitocore_mouse.reactions.HDCAtr.upper_bound=(0.09+0.043)*0.89


In [26]:
mitocore_mouse.objective=mitocore_mouse.reactions.OF_ATP_MitoCore
solution=mitocore_mouse.optimize()
print(solution.objective_value)
healthy_model_objective_value=solution.objective_value
compteur=0
for f in solution.fluxes :
    if f != 0 :        
        compteur+=1
print("Nombre de réactions utilisées pour maximiser la production d'ATP : "+str(compteur))  
print(get_fluxes_from_mitocore_metabolite(mitocore_mouse.metabolites.accoa_m))
print("Quotient respiratoire : ")
print(get_mitocore_respiratory_exchange_ratio(mitocore_mouse))
print(mitocore_mouse.metabolites.accoa_m.summary())
print(mitocore_mouse.metabolites.co2_c.summary())
print(mitocore_mouse.reactions.LDH_L.summary())
print(mitocore_mouse.reactions.O2t.summary())

33.89097240494641
Nombre de réactions utilisées pour maximiser la production d'ATP : 292
{'Isoleucine degradation': 0.17, 'FA and ketone body metabolism / Ketogenesis': 31.09, 'Ketogenesis / Leucine degradation': 0.68, 'FA metabolism': 30.6, 'TCA cycle': 37.45}
99.98999999999998
Quotient respiratoire : 
o2 : 6.796790000000127
Co2 : 6.076450000000001
0.8940176171398393
accoa_m
Formula: C23H34N7O17P3S

Producing Reactions
-------------------
Percent   Flux        Reaction                                                                  Definition
  0.17%  0.004        ACACT10m                                     2maacoa_m + coa_m --> accoa_m + ppcoa_m
 31.09% 0.7292        ACACT1rm                                             2.0 accoa_m <=> aacoa_m + coa_m
  0.68%  0.016           HMGLm                                               hmgcoa_m --> acac_m + accoa_m
  5.10% 0.1196 MTPC14_MitoCore 3tetd7ecoa_m + coa_m + h2o_m + nad_m <=> accoa_m + ddcacoa_m + h_m + nadh_m
  5.10% 0.1196 MTPC16

In [27]:
#65% VO2max
#coeur
#mitocore_mouse.reactions.GLCt1r.upper_bound=2.027
#mitocore_mouse.reactions.HDCAtr.upper_bound=2.400
#muscle
mitocore_mouse.reactions.GLCt1r.upper_bound=2.33*(0.09+0.043)*0.45807
mitocore_mouse.reactions.HDCAtr.upper_bound=2.33*(0.09+0.043)*0.54193
print(mitocore_mouse.reactions.GLCt1r.upper_bound)
print(mitocore_mouse.reactions.HDCAtr.upper_bound)

0.1419513123
0.1679386877


In [28]:
mitocore_mouse.objective=mitocore_mouse.reactions.OF_ATP_MitoCore
solution=mitocore_mouse.optimize()
print(solution.objective_value)
healthy_model_objective_value=solution.objective_value
compteur=0
for f in solution.fluxes :
    if f != 0 :        
        compteur+=1
print("Nombre de réactions utilisées pour maximiser la production d'ATP : "+str(compteur))  
print(get_fluxes_from_mitocore_metabolite(mitocore_mouse.metabolites.accoa_m))
print("Quotient respiratoire : ")
print(get_mitocore_respiratory_exchange_ratio(mitocore_mouse))
print(mitocore_mouse.metabolites.accoa_m.summary())
print(mitocore_mouse.metabolites.pmtcrn_m.summary())
print(mitocore_mouse.metabolites.glc_D_e.summary())
print(mitocore_mouse.reactions.LDH_L.flux)
print(mitocore_mouse.reactions.O2t.summary())

43.63811597662915
Nombre de réactions utilisées pour maximiser la production d'ATP : 307
{'Isoleucine degradation': 0.13, 'FA and ketone body metabolism / Ketogenesis': 27.65, 'Ketogenesis / Leucine degradation': 0.53, 'FA metabolism': 33.9, 'TCA cycle': 37.81}
100.02000000000002
Quotient respiratoire : 
o2 : 8.700797690899707
Co2 : 7.633476876999999
0.8773306940562433
accoa_m
Formula: C23H34N7O17P3S

Producing Reactions
-------------------
Percent   Flux        Reaction                                                                  Definition
  0.13%  0.004        ACACT10m                                     2maacoa_m + coa_m --> accoa_m + ppcoa_m
 27.65% 0.8284        ACACT1rm                                             2.0 accoa_m <=> aacoa_m + coa_m
  0.53%  0.016           HMGLm                                               hmgcoa_m --> acac_m + accoa_m
  5.65% 0.1692 MTPC14_MitoCore 3tetd7ecoa_m + coa_m + h2o_m + nad_m <=> accoa_m + ddcacoa_m + h_m + nadh_m
  5.65% 0.1692 MTPC1

In [33]:
#exercice à 85%
#coeur
#mitocore_mouse.reactions.GLCt1r.upper_bound=4.895
#mitocore_mouse.reactions.HDCAtr.upper_bound=1.582
#muscle
mitocore_mouse.reactions.GLCt1r.upper_bound=3.41*(0.09+0.043)*0.75562
mitocore_mouse.reactions.HDCAtr.upper_bound=3.41*(0.09+0.043)*0.24438

In [32]:
mitocore_mouse.objective=mitocore_mouse.reactions.OF_ATP_MitoCore
solution=mitocore_mouse.optimize()
print(solution.objective_value)
healthy_model_objective_value=solution.objective_value
compteur=0
for f in solution.fluxes :
    if f != 0 :        
        compteur+=1
print("Nombre de réactions utilisées pour maximiser la production d'ATP : "+str(compteur))  
print(get_fluxes_from_mitocore_metabolite(mitocore_mouse.metabolites.accoa_m))
print("Quotient respiratoire : ")
print(get_mitocore_respiratory_exchange_ratio(mitocore_mouse))
print(mitocore_mouse.metabolites.accoa_m.summary())
print(mitocore_mouse.reactions.LDH_L.summary())
print(mitocore_mouse.reactions.O2t.summary())

47.39470009437215
Nombre de réactions utilisées pour maximiser la production d'ATP : 301
{'Isoleucine degradation': 0.12, 'FA and ketone body metabolism / Ketogenesis': 31.38, 'Ketogenesis / Leucine degradation': 0.49, 'FA metabolism': 20.7, 'TCA cycle': 47.28}
99.97000000000001
Quotient respiratoire : 
o2 : 9.275852243799852
Co2 : 8.532266613999996
0.9198364085308838
accoa_m
Formula: C23H34N7O17P3S

Producing Reactions
-------------------
Percent   Flux        Reaction                                                                  Definition
  0.12%  0.004        ACACT10m                                     2maacoa_m + coa_m --> accoa_m + ppcoa_m
 31.38%  1.018        ACACT1rm                                             2.0 accoa_m <=> aacoa_m + coa_m
  0.49%  0.016           HMGLm                                               hmgcoa_m --> acac_m + accoa_m
  3.45% 0.1121 MTPC14_MitoCore 3tetd7ecoa_m + coa_m + h2o_m + nad_m <=> accoa_m + ddcacoa_m + h_m + nadh_m
  3.45% 0.1121 MTPC16

In [19]:
print(mitocore_mouse.metabolites.glc_D_c.summary())
print(mitocore_mouse.metabolites.g3p_c.summary())
print(mitocore_mouse.metabolites.get_by_id("pyr_c").summary()) # à partir d'un glc on fait 4 pyr
print(mitocore_mouse.metabolites.lac_L_c.summary())

glc_D_c
Formula: C6H12O6

Producing Reactions
-------------------
Percent   Flux Reaction          Definition
100.00% 0.3427   GLCt1r glc_D_e <=> glc_D_c

Consuming Reactions
-------------------
Percent    Flux Reaction                              Definition
100.00% -0.3427     HEX1 atp_c + glc_D_c --> adp_c + g6p_c + h_c
g3p_c
=====
Formula: C3H5O6P

Producing Reactions
-------------------
Percent   Flux Reaction               Definition
 49.28% 0.3427      FBA fdp_c <=> dhap_c + g3p_c
 50.72% 0.3527      TPI         dhap_c <=> g3p_c

Consuming Reactions
-------------------
Percent    Flux Reaction                                      Definition
100.00% -0.6954     GAPD g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
pyr_c
=====
Formula: C3H3O3

Producing Reactions
-------------------
Percent   Flux Reaction                               Definition
  1.27%  0.017  ALATA_L      akg_c + ala_L_c <=> glu_L_c + pyr_c
 42.98%  0.575    LDH_L lac_L_c + nad_c <=> h_c + nadh_c + pyr_c
 55.75%

## Brute force

In [7]:
list_reaction_to_test=[mitocore_mouse.reactions.GLCt1r,mitocore_mouse.reactions.HDCAtr]
range_params=np.arange(0,2,0.01)
dict_result=dict()
mitocore_mouse.objective=mitocore_mouse.reactions.OF_ATP_MitoCore
def test_range_on_reaction(list_r_to_test,indice,prior_param=None):
    """
    fonction récursive qui créée un dictionnaire dict_result : 
    le dictionnaire a pour première clé le parametre de la première réactions de 
    la liste list_reaction_to_test
    la deuxième clé est le parametre de la 2eme reactions de la liste
    
    cela aboutit à la création d'une matrixe de dimension NxN N étant le nombre
    de réaction dans list_reactions_to_test
    à chaque case de la matrice est associé la valeur renvoyé par la fba avec lesdits parametres
    
    """
    print(list_r_to_test[indice])
    for param in range_params :
        #print("param")
        #print(param)
        #print("prior_param")
        #print(prior_param)
        
        if param not in dict_result.keys():
            dict_result[param]=dict()
        
        if prior_param!=None: 
            if param not in dict_result[prior_param].keys() :
                mitocore_mouse.reactions.GLCt1r.upper_bound=prior_param
                mitocore_mouse.reactions.HDCAtr.upper_bound=param
                solution=mitocore_mouse.optimize()
                dict_result[prior_param][param]=solution.objective_value
        #print("dict")
        #print(dict_result)
        try :
            test_range_on_reaction(list_r_to_test,indice+1,prior_param=param)
        except IndexError :
            pass
    return dict_result
test_range_on_reaction(list_reaction_to_test,0)


GLCt1r: glc_D_e <-- glc_D_c
HDCAtr: hdca_e <-- hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hdca_e <=> hdca_c
HDCAtr: hd

{0.0: {0.0: 20.17640158355156,
  0.01: 21.29418704133702,
  0.02: 22.411972499122477,
  0.03: 23.529757956907936,
  0.04: 24.647543414693388,
  0.05: 25.76532887247884,
  0.06: 26.883114330264316,
  0.07: 28.00089978804977,
  0.08: 29.11868524583523,
  0.09: 30.23647070362068,
  0.1: 31.354256161406134,
  0.11: 32.47204161919159,
  0.12: 33.58982707697705,
  0.13: 34.70761253476251,
  0.14: 35.82539799254796,
  0.15: 36.94318345033339,
  0.16: 38.06096890811888,
  0.17: 39.178754365904325,
  0.18: 40.29653982368977,
  0.19: 41.41432528147524,
  0.2: 42.532110739260695,
  0.21: 43.649896197046154,
  0.22: 44.76768165483161,
  0.23: 45.885467112617064,
  0.24: 47.00325257040254,
  0.25: 48.121038028187975,
  0.26: 49.23882348597346,
  0.27: 50.356608943758914,
  0.28: 51.47439440154436,
  0.29: 52.592179859329825,
  0.3: 53.70996531711527,
  0.31: 54.82775077490072,
  0.32: 55.94553623268618,
  0.33: 57.063321690471625,
  0.34: 58.18110714825708,
  0.35000000000000003: 59.298892606042564

In [12]:
for param1 in dict_result.keys() :
    for param2 in dict_result[param1].keys() :
        atp_production=dict_result[param1][param2]
        if atp_production>22.5 and atp_production<25 :
            print(str(param1)+","+str(param2)+" : "+str(dict_result[param1][param2]))

0.0,0.03 : 23.529757956907936
0.0,0.04 : 24.647543414693388
0.01,0.02 : 22.742351417501418
0.01,0.03 : 23.86013687528687
0.01,0.04 : 24.97792233307231
0.02,0.02 : 23.072730335880344
0.02,0.03 : 24.19051579366579
0.03,0.02 : 23.403109254259256
0.03,0.03 : 24.520894712044704
0.04,0.01 : 22.615702714852713
0.04,0.02 : 23.733488172638165
0.04,0.03 : 24.851273630423627
0.05,0.01 : 22.946081633231625
0.05,0.02 : 24.06386709101707
0.06,0.01 : 23.276460551610544
0.06,0.02 : 24.394246009396003
0.07,0.01 : 23.606839469989456
0.07,0.02 : 24.72462492777491
0.08,0.0 : 22.819432930582913
0.08,0.01 : 23.93721838836838
0.09,0.0 : 23.14981184896185
0.09,0.01 : 24.267597306747312
0.1,0.0 : 23.480190767340762
0.1,0.01 : 24.597976225126224
0.11,0.0 : 23.810569685719692
0.11,0.01 : 24.928355143505144
0.12,0.0 : 24.14094860409863
0.13,0.0 : 24.471327522477548
0.14,0.0 : 24.801706440856474


In [14]:
#essayer de trouver la quantité d'oxygène correcte
mitocore_mouse=cobra.io.read_sbml_model("Models/Mitocore_mouse.xml")

In [71]:
mitocore_mouse.objective=mitocore_mouse.reactions.OF_ATP_MitoCore
mitocore_mouse.reactions.GLCt1r.upper_bound=(0.09+0.043)*0.11
mitocore_mouse.reactions.HDCAtr.upper_bound=(0.09+0.043)*0.89

for Cox in np.arange(4,18,0.05) :
    mitocore_mouse.reactions.O2t.upper_bound=Cox
    solution=mitocore_mouse.optimize()
    if solution.objective_value>20 and solution.objective_value<25 or mitocore_mouse.reactions.LDH_L.flux<0:
        print(Cox)
        print(solution)
        print(mitocore_mouse.reactions.LDH_L.flux)
#Si l'entrée d'oxygène est entre 4 et 4.5 alors on obtient des valeurs d'ATP cohérente :

4.0
<Solution 20.672 at 0x7f86dd99a860>
0.575
4.05
<Solution 20.915 at 0x7f86dd99bc40>
0.575
4.1
<Solution 21.158 at 0x7f86de55cf70>
0.575
4.1499999999999995
<Solution 21.401 at 0x7f86dd998dc0>
0.575
4.199999999999999
<Solution 21.644 at 0x7f86ddc1ae00>
0.575
4.249999999999999
<Solution 21.887 at 0x7f86dd998c70>
0.575
4.299999999999999
<Solution 22.130 at 0x7f86dd9e8280>
0.575
4.349999999999999
<Solution 22.373 at 0x7f86dd9981f0>
0.575
4.399999999999999
<Solution 22.616 at 0x7f86dd5cb400>
0.575
4.449999999999998
<Solution 22.859 at 0x7f86dd99af80>
0.575
4.499999999999998
<Solution 23.102 at 0x7f86dd5cae60>
0.575
4.549999999999998
<Solution 23.345 at 0x7f86dd998dc0>
0.575
4.599999999999998
<Solution 23.588 at 0x7f86ddc4f340>
0.575
4.649999999999998
<Solution 23.831 at 0x7f86dd99aef0>
0.575
4.6999999999999975
<Solution 24.074 at 0x7f86de520400>
0.575
4.749999999999997
<Solution 24.317 at 0x7f86ddc56650>
0.575
4.799999999999997
<Solution 24.560 at 0x7f86dd9988e0>
0.575
4.849999999999997
<

In [9]:
mitocore_mouse=cobra.io.read_sbml_model("Models/Mitocore_mouse.xml")
#25%
#mitocore_mouse.reactions.GLCt1r.upper_bound=(0.09+0.043)*0.11
#mitocore_mouse.reactions.HDCAtr.upper_bound=(0.09+0.043)*0.89
#65%
#mitocore_mouse.reactions.GLCt1r.upper_bound=2.33*(0.09+0.043)*0.45807
#mitocore_mouse.reactions.HDCAtr.upper_bound=2.33*(0.09+0.043)*0.54193
#85%
#mitocore_mouse.reactions.GLCt1r.upper_bound=3.41*(0.09+0.043)*0.75562
#mitocore_mouse.reactions.HDCAtr.upper_bound=3.41*(0.09+0.043)*0.24438
mitocore_mouse.reactions.O2t.upper_bound=4.5
solution=mitocore_mouse.optimize()
print(solution.objective_value)
healthy_model_objective_value=solution.objective_value
compteur=0
for f in solution.fluxes :
    if f != 0 :        
        compteur+=1
print("Nombre de réactions utilisées pour maximiser la production d'ATP : "+str(compteur))  
print(get_fluxes_from_mitocore_metabolite(mitocore_mouse.metabolites.accoa_m))
print("Quotient respiratoire : ")
print(get_mitocore_respiratory_exchange_ratio(mitocore_mouse))

23.59535285184566
Nombre de réactions utilisées pour maximiser la production d'ATP : 219
{'Isoleucine degradation': 0.26, 'FA and ketone body metabolism / Ketogenesis': 27.2, 'Ketogenesis / Leucine degradation': 1.04, 'FA metabolism': 0.8400000000000001, 'TCA cycle': 70.65}
99.99000000000001
Quotient respiratoire : 
o2 : 4.5
Co2 : 4.60904152680869
1.0242314504019312
